In [29]:
%pip install -qU langchain_google_vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.3 MB/s eta 0:00:00


In [30]:
from langchain_google_vertexai import VertexAIEmbeddings
import time
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [31]:
PROJECT_ID = "langgraph-465507"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [32]:
from langchain_google_vertexai import VertexAIEmbeddings

# Initialize the a specific Embeddings Model version
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [33]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

'LangChain is the framework for building context-aware reasoning applications'

In [45]:
def get_vembedding_with_retry(text, max_retries=5, initial_delay=1):
    """Gets an embedding with retry logic."""
    for attempt in range(max_retries):
        try:
            llm = VertexAIEmbeddings(model_name="text-embedding-004")
            embedding = llm.embed_query(text)
            return embedding
        except Exception as e:
            if "Deadline Exceeded" in str(e):
                print(f"Attempt {attempt + 1} failed: Deadline Exceeded. Retrying in {initial_delay} seconds...")
                time.sleep(initial_delay)
                initial_delay *= 2  # Exponential backoff
            else:
                print(f"Attempt {attempt + 1} failed with a different error: {e}")
                raise  # Re-raise the exception if it's not a timeout

    raise Exception("Max retries reached. Embedding failed after multiple attempts.")


In [46]:
# Example usage:
try:
    text_to_embed = "This is a sample text for embedding."
    embedding_vector = get_vembedding_with_retry(text_to_embed)
    print("Embedding:", embedding_vector)
except Exception as e:
    print("Error getting embedding:", e)

Embedding: [-0.01538902334868908, 0.038881223648786545, -0.08773738890886307, 0.03219539672136307, 0.05101846903562546, 0.008384954184293747, 0.048795174807310104, 0.04102547839283943, -0.0227595753967762, -0.020966939628124237, -0.004620279185473919, 0.07042087614536285, 0.06501758098602295, 0.012442711740732193, 0.009931196458637714, -0.03578798100352287, 0.006560828071087599, -0.011659456416964531, -0.1104140654206276, 0.004329781513661146, 0.018862804397940636, -0.01856529526412487, 0.014777150005102158, -0.06451578438282013, -0.009657773189246655, -0.046977847814559937, 0.0719790980219841, -0.005070679355412722, 0.02242405153810978, 0.0007472199504263699, 0.005954102147370577, 0.055182892829179764, 0.03505053371191025, -0.01819041557610035, 0.0009941422613337636, -0.013366593047976494, 0.030651096254587173, -0.007961856201291084, 0.0434538833796978, -0.031064484268426895, -0.04189904034137726, 0.033246103674173355, -0.032951001077890396, -0.0006105987704358995, -0.0193372014909982